In [66]:
import numpy as np
import pandas as pd
import statistics as st
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

In [67]:
#Load CSV
data = pd.read_csv("train.csv")
data2 = pd.read_csv("Test.csv") 
print(data.shape)

(8523, 12)


In [68]:
data.head(20)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [69]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [70]:
missing_values_count = data.isnull().sum()
missing_values_count[0:10]

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
dtype: int64

In [71]:
total_cells = np.product(data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

3.786812155344362


In [72]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [73]:
# data['Item_Weight'] = data['Item_Weight'].fillna(0)
# data.head(10)

In [74]:
data.mode(axis='columns', numeric_only = 'True')

,0,1,2,3,4
0,0.016047,9.300,249.8092,1999.0000,3735.1380
1,0.019278,5.920,48.2692,443.4228,2009.0000
2,0.016760,17.500,141.6180,1999.0000,2097.2700
3,0.000000,19.200,182.0950,732.3800,1998.0000
4,0.000000,8.930,53.8614,994.7052,1987.0000
...,...,...,...,...,...
8518,0.056783,6.865,214.5218,1987.0000,2778.3834
8519,0.046982,8.380,108.1570,549.2850,2002.0000
8520,0.035186,10.600,85.1224,1193.1136,2004.0000
8521,0.145221,7.210,103.1332,1845.5976,2009.0000


In [75]:
df = st.mode(data['Item_Weight'])
print("Mode is :",df)
data['Item_Weight'] = data['Item_Weight'].fillna(0)
data.head(10)

Mode is : 12.15


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,0.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [76]:
#Converting Outlet_Size into Numarical data
data['Outlet_Size'].replace(['Medium', 'High','Small'],[0, 1,2], inplace=True)
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,0.0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,0.0,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,0.0,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,1.0,Tier 3,Supermarket Type1,994.7052


In [77]:
dt = st.mode(data['Outlet_Size'])
print("Mode is :",dt)
data['Outlet_Size'] = data['Outlet_Size'].fillna(0)
data.head(10)

Mode is : 0.0


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,0.0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,0.0,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,0.0,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,0.0,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,1.0,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,0.0,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,1.0,Tier 3,Supermarket Type1,343.5528
7,FDP10,0.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,0.0,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,0.0,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,0.0,Tier 2,Supermarket Type1,4710.5350


In [78]:
#Converting Outlet_Size into Numarical data
#data['Item_Fat_Content'].replace(['Low Fat', 'Regular'],[0, 1], inplace=True)
data['Outlet_Location_Type'].replace(['Tier 1', 'Tier 2','Tier 3'],[0, 1, 2], inplace=True)
data['Outlet_Type'].replace(['Supermarket Type1', 'Supermarket Type2','Supermarket Type3','Grocery Store'],[0, 1, 2, 3], inplace=True)
# data['Item_Type'].replace(['Meat', 'Soft Drinks','Dairy','Fruits and Vegetables','Household',''],[0, 1], inplace=True)
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,0.0,0,0,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,0.0,2,1,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,0.0,0,0,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,0.0,2,3,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,1.0,2,0,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,0.0,2,1,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,1.0,2,0,343.5528
7,FDP10,0.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,0.0,2,2,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,0.0,1,0,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,0.0,1,0,4710.5350


In [79]:
data.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [80]:
#Univariate Analysis
import plotly.express as pl
grp = pl.histogram(data['Item_Weight'])
grp.show()

In [103]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace('LF', 'Low Fat')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace('low fat', 'Low Fat')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace('reg', 'Regular')

In [104]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [105]:
fig = pl.histogram(data['Item_Fat_Content'])
fig.show()

In [106]:
fig = pl.histogram(data['Item_Type'])
fig.show()

In [107]:
data['Outlet_Establishment_Year'].value_counts()

1985    1463
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64

In [108]:
encoder = LabelEncoder()

In [117]:
data['Item_Identifier'] = encoder.fit_transform(data['Item_Identifier'])
data['Outlet_Identifier'] = encoder.fit_transform(data['Outlet_Identifier'])
data['Item_Fat_Content'] = encoder.fit_transform(data['Item_Fat_Content'])
data['Item_Type'] = encoder.fit_transform(data['Item_Type'])

In [118]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,0,0.016047,4,249.8092,9,1999,0.0,0,0,3735.1380
1,8,5.92,1,0.019278,14,48.2692,3,2009,0.0,2,1,443.4228
2,662,17.50,0,0.016760,10,141.6180,9,1999,0.0,0,0,2097.2700
3,1121,19.20,1,0.000000,6,182.0950,0,1998,0.0,2,3,732.3800
4,1297,8.93,0,0.000000,9,53.8614,1,1987,1.0,2,0,994.7052


In [119]:
x = data.drop(columns = 'Item_Outlet_Sales',axis = 1)
y = data["Item_Outlet_Sales"]

In [120]:
x

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,156,9.300,0,0.016047,4,249.8092,9,1999,0.0,0,0
1,8,5.920,1,0.019278,14,48.2692,3,2009,0.0,2,1
2,662,17.500,0,0.016760,10,141.6180,9,1999,0.0,0,0
3,1121,19.200,1,0.000000,6,182.0950,0,1998,0.0,2,3
4,1297,8.930,0,0.000000,9,53.8614,1,1987,1.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...
8518,370,6.865,0,0.056783,13,214.5218,1,1987,1.0,2,0
8519,897,8.380,1,0.046982,0,108.1570,7,2002,0.0,1,0
8520,1357,10.600,0,0.035186,8,85.1224,6,2004,2.0,1,0
8521,681,7.210,1,0.145221,13,103.1332,3,2009,0.0,2,1


In [121]:
y

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [122]:
#Training Dataset Prediction 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=2)
print(x.shape,x_train.shape,x_test.shape)

(8523, 11) (5966, 11) (2557, 11)


In [123]:
model = XGBRegressor(n_estimators = 750,learning_rate = 0.007)

In [124]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.007, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=750, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [125]:
train_data_prediction = model.predict(x_train)

In [126]:
r2_train = metrics.r2_score(y_train,train_data_prediction)
print("R Squared Value : ",r2_train)

R Squared Value :  0.6936404351099061


In [128]:
#testing dataset
data2.shape

(5681, 11)

In [129]:
data2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [130]:
data2.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
count,4705.000000,5681.000000,5681.000000,5681.000000
mean,12.695633,0.065684,141.023273,1997.828903
std,4.664849,0.051252,61.809091,8.372256
min,4.555000,0.000000,31.990000,1985.000000
25%,8.645000,0.027047,94.412000,1987.000000
50%,12.500000,0.054154,141.415400,1999.000000
75%,16.700000,0.093463,186.026600,2004.000000
max,21.350000,0.323637,266.588400,2009.000000


In [131]:
data2.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [132]:
data2["Item_Weight"].fillna(data2["Item_Weight"].mean(),inplace=True)
data2.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [136]:
data2['Outlet_Size'].replace(['Medium', 'High','Small'],[0, 1,2], inplace=True)
d2 = st.mode(data['Outlet_Size'])
print("Mode is :",d2)
data2['Outlet_Size'] = data2['Outlet_Size'].fillna(d2)

Mode is : 0.0


In [137]:
data2.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750000,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,0.0,Tier 1,Supermarket Type1
1,FDW14,8.300000,reg,0.038428,Dairy,87.3198,OUT017,2007,0.0,Tier 2,Supermarket Type1
2,NCN55,14.600000,Low Fat,0.099575,Others,241.7538,OUT010,1998,0.0,Tier 3,Grocery Store
3,FDQ58,7.315000,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,0.0,Tier 2,Supermarket Type1
4,FDY38,12.695633,Regular,0.118599,Dairy,234.2300,OUT027,1985,0.0,Tier 3,Supermarket Type3
5,FDH56,9.800000,Regular,0.063817,Fruits and Vegetables,117.1492,OUT046,1997,2.0,Tier 1,Supermarket Type1
6,FDL48,19.350000,Regular,0.082602,Baking Goods,50.1034,OUT018,2009,0.0,Tier 3,Supermarket Type2
7,FDC48,12.695633,Low Fat,0.015782,Baking Goods,81.0592,OUT027,1985,0.0,Tier 3,Supermarket Type3
8,FDN33,6.305000,Regular,0.123365,Snack Foods,95.7436,OUT045,2002,0.0,Tier 2,Supermarket Type1
9,FDA36,5.985000,Low Fat,0.005698,Baking Goods,186.8924,OUT017,2007,0.0,Tier 2,Supermarket Type1


In [138]:
data2.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [139]:
data2.replace({'FatContent': {'low fat':'Low Fat','LF':'Low Fat','reg':'Regular'}},inplace=True)

In [141]:
encoder = LabelEncoder()
data2['Item_Fat_Content'] = encoder.fit_transform(data2['Item_Fat_Content'])
data2['Item_Type'] = encoder.fit_transform(data2['Item_Type'])
data2['Outlet_Location_Type'] = encoder.fit_transform(data2['Outlet_Location_Type'])
data2['Outlet_Type'] = encoder.fit_transform(data2['Outlet_Type'])
data2['Outlet_Identifier'] = encoder.fit_transform(data2['Outlet_Identifier'])
data2['Item_Identifier'] = encoder.fit_transform(data2['Item_Identifier'])

In [142]:
data2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1103,20.750000,1,0.007565,13,107.8622,9,1999,0.0,0,1
1,1067,8.300000,4,0.038428,4,87.3198,2,2007,0.0,1,1
2,1406,14.600000,1,0.099575,11,241.7538,0,1998,0.0,2,0
3,809,7.315000,1,0.015388,13,155.0340,2,2007,0.0,1,1
4,1184,12.695633,2,0.118599,4,234.2300,5,1985,0.0,2,3


In [146]:
model.fit(x,y)
test_p = model.predict(data2)
print(test_p)

[1570.117  1297.0265 4044.1812 ... 1869.7727 3610.123  1255.1915]


In [149]:
Item_Outlet_Sales = test_p
ds = pd.DataFrame(Item_Outlet_Sales, columns = ['Item_Outlet_Sales'])

data2['Item_Outlet_Sales'] = Item_Outlet_Sales
data2.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,1103,20.750000,1,0.007565,13,107.8622,9,1999,0.0,0,1,1570.116943
1,1067,8.300000,4,0.038428,4,87.3198,2,2007,0.0,1,1,1297.026489
2,1406,14.600000,1,0.099575,11,241.7538,0,1998,0.0,2,0,4044.181152
3,809,7.315000,1,0.015388,13,155.0340,2,2007,0.0,1,1,2476.358154
4,1184,12.695633,2,0.118599,4,234.2300,5,1985,0.0,2,3,669.948730
